# Data Querying

1. Write the query to print the hacker id, name of the hacker who has the most submissions. If more
than one such hacker has the maximum submission number, the results are sorted by ascending hack id.

```
SELECT
	h.NAME,
	h.hacker_id
FROM
	Hackers h
	JOIN Submissions s ON h.hacker_id = s.hacker_id
GROUP BY
	h.hacker_id
HAVING
	count( s.submission_id ) >= (
	SELECT
		count( s2.submission_id )
	FROM
		Hackers h2
		JOIN Submissions s2 ON h2.hacker_id = s2.hacker_id
	GROUP BY
		h2.hacker_id
	ORDER BY
		count( s2.submission_id ) DESC
		LIMIT 1
	)
ORDER BY
	h.hacker_id ASC;
```

2. Write the query to print the task id, task descriptionand the number of submissions received
of the task which receives the most submissions. If there are more than one such task, the results are sorted by
the ascending task id.

```
-- t2
SELECT
	t.task_id,
	t.task_description,
	count( s.submission_id ) AS count
FROM
	Submissions s
	JOIN tasks t ON t.task_id = s.task_id
GROUP BY
	t.task_id
HAVING
	count >= (
	SELECT
		count( s2.submission_id )
	FROM
		Submissions s2,
		tasks t2
	WHERE
		t2.task_id = s2.task_id
	GROUP BY
		t2.task_id
	ORDER BY
		count( s2.submission_id ) DESC
		LIMIT 1
	)
ORDER BY
	t.task_id ASC;
```

3. Write the query to print the submission id, name, task description of the submission whose score
is the highest one among all scores obtained on 2022-10-01. If more than one such submissions, the results are
sorted by ascending submission id.

```
SELECT
	s.submission_id,
	h.`name`,
	t.task_description
FROM
	Submissions s
	JOIN tasks t ON t.task_id = s.task_id
	JOIN hackers h ON h.hacker_id = s.hacker_id
WHERE
	s.submission_data = '2022-10-01'
	AND 1 >= ( SELECT count( DISTINCT s2.score ) FROM Submissions s2 WHERE s.score <= s2.score )
ORDER BY
	s.submission_id ASC;
```

4. Write the query to print the hacker id, name of the hacker who has the highest total scores on all
tasks. For each task, if the hacker has multiple submissions, only the best score is accounted. If more than one
such hacker, the results are sorted by descending hacker id.

```
-- t4
SELECT
	h.hacker_id,
	h.`name`
FROM
	Submissions s
	JOIN tasks t ON t.task_id = s.task_id
	JOIN hackers h ON h.hacker_id = s.hacker_id
where
	1 >= ( SELECT count( DISTINCT s2.score ) FROM Submissions s2 WHERE s.score <= s2.score )
ORDER BY
	h.hacker_id ASC;
```

5. Write the query to print the hacker id, name, bonus of the hackers and the task fulfilling the
requirement that the hackers win the bonus on task with task id=25.

```
SELECT
	h2.hacker_id, h2.`NAME`, t2.bonus
FROM
	Submissions s2
	JOIN tasks t2 ON t2.task_id = s2.task_id
	JOIN hackers h2 ON h2.hacker_id = s2.hacker_id
WHERE
	t2.task_id = 25
GROUP BY
	h2.hacker_id
ORDER BY
	max( s2.score ) DESC
```

6. Write the query to print the hacker id,
hacker name, bank account and the total bonus of all participants. The result is sorted by descending total
bonus and ascending hacker id. Exclude all hackers with a total bonus of 0 from the result.

# Question2

1. Please write two basic approaches for recommender system and briefly explain them.

One is User-based Collaborative filtering which finds a subset of users who have similar tastes and preferences to the target user and use this subset for offering recommendations. It selects top-k nearest neighbors of every user using similarity matrix and predict the scores of the users with prediction equation.

Another is Matrix factorization method which factorizes the matrix into a multiplication of two, each row/column will represent the user/item embeddings. We can get the prediction of a rating of an item by the calculation of the dot product of user and item matrix.

2. Cold start is a severe problem for recommender system. Please explain what is cold start problem in recommender system and how to solve it (list at least TWO methods).

There are three cases of cold start.
1. New community: refers to the start-up of the recommender, when, although a catalogue of items might exist, almost no users are present and the lack of user interaction makes it very hard to provide reliable recommendations
2. New item: a new item is added to the system, it might have some content information but no interactions are present
3. New user: a new user registers and has not provided any interaction yet, therefore it is not possible to provide personalized recommendations

Solution:
1. Profile completion: One of the available options when dealing with cold users or items is to rapidly acquire some preference data. The cold start problem would imply that the user has to dedicate an amount of effort using the system in its 'dumb' state – contributing to the construction of their user profile – before the system can start providing any intelligent recommendations.
2. Feature mapping: One example of this approaches is called attribute to feature mapping.A matrix factorization model represents the user-item interactions as the product of two rectangular matrices whose content is learned using the known interactions via machine learning. Each user will be associated to a row of the first matrix and each item with a column of the second matrix. The row or column associated to a specific user or item is called latent factors.When a new item is added it has no associated latent factors and the lack of interactions does not allow to learn them, as it was done with other items. If each item is associated to some features (e.g. author, year, publisher, actors) it is possible to define an embedding function, which given the item features estimates the corresponding item latent factors.

3. Rating prediction is an important task for the recommender system. Try to implement a recom-
mendation model on the Movielens-100k dataset to predict user rating. There are some instructions as following:
(a) You can download the dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip
(b) You just need to train on ua.base and evaluate on ua.test.
(c) You can use methods such as matrix factorization, two tower models, or graph-based models.
(d) For basic requirements, you can only use the rating file in the dataset. To obtain richer features of user/item,
you can also use the corresponding user/item information in the dataset.

In [2]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as MAE
import os
from tqdm import tqdm

### Hyper-parameters setting

In [8]:
staticDir = 'ml-100k'

### 1. Data loading and training/test set split

In [72]:
def get_data(path):
    data_item_list = []
    for data_item in open(path):
        # data_item_list：[(6040, 858, 4, 956703932), (1, 593, 3, 1112484661), ...]
        temp_tuple = list(data_item.strip().split()[:4])  # delimiter
        temp_tuple[0] = int(temp_tuple[0])  # user ID
        temp_tuple[1] = int(temp_tuple[1])  # item ID
        temp_tuple[2] = int(temp_tuple[2])  # item rating，
        temp_tuple[3] = int(temp_tuple[3])  # timestamp
        data_item_list.append(tuple(temp_tuple))
    # We order data_item_list based on the timestamp and the user ID
    data_item_list = sorted(data_item_list, key=lambda tup: tup[3])
    data_item_list = sorted(data_item_list, key=lambda tup: tup[0])
    return data_item_list


data_item_list = get_data(staticDir + '/ua.base')
data_item_list

[(1, 168, 5, 874965478),
 (1, 172, 5, 874965478),
 (1, 165, 5, 874965518),
 (1, 156, 4, 874965556),
 (1, 166, 5, 874965677),
 (1, 196, 5, 874965677),
 (1, 187, 4, 874965678),
 (1, 14, 5, 874965706),
 (1, 127, 5, 874965706),
 (1, 250, 4, 874965706),
 (1, 109, 5, 874965739),
 (1, 181, 5, 874965739),
 (1, 1, 5, 874965758),
 (1, 246, 5, 874965905),
 (1, 50, 5, 874965954),
 (1, 248, 4, 874965954),
 (1, 257, 4, 874965954),
 (1, 249, 4, 874965970),
 (1, 253, 5, 874965970),
 (1, 262, 3, 875071421),
 (1, 93, 5, 875071484),
 (1, 124, 5, 875071484),
 (1, 224, 5, 875071484),
 (1, 19, 5, 875071515),
 (1, 123, 4, 875071541),
 (1, 137, 5, 875071541),
 (1, 7, 4, 875071561),
 (1, 146, 4, 875071561),
 (1, 235, 5, 875071589),
 (1, 15, 5, 875071608),
 (1, 24, 3, 875071713),
 (1, 126, 2, 875071713),
 (1, 245, 2, 875071713),
 (1, 260, 1, 875071713),
 (1, 264, 2, 875071713),
 (1, 237, 2, 875071749),
 (1, 13, 5, 875071805),
 (1, 25, 4, 875071805),
 (1, 121, 4, 875071823),
 (1, 251, 4, 875071843),
 (1, 236, 4,

In [73]:
def getUIMat(data):
    # build U-I rating matrix
    user_list = [i[0] for i in data]
    item_list = [i[1] for i in data]
    UI_matrix = np.zeros((max(user_list) + 1, max(item_list) + 1))
    for each_interaction in tqdm(data, total=len(data)):
        UI_matrix[each_interaction[0]][each_interaction[1]] = each_interaction[2]
    return UI_matrix


R = getUIMat(data_item_list)
R

100%|██████████| 90570/90570 [00:00<00:00, 1932199.31it/s]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 3., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       ...,
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 5., ..., 0., 0., 0.]])

### 2.Matrix factorization model building

In [74]:
class MF():
    def __init__(self, R, K, alpha, beta, iterations):
        """
        parameters
        - R (ndarray)   : user-item rating matrix
        - K (int)       : dimension of hidden features
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        - beta (int)    : number of iterations
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize the hidden user-feature matrix and item-feature matrix
        self.P = np.random.normal(scale=1. / self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1. / self.K, size=(self.num_items, self.K))
        # Building training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Iterate the gradient descent to update matrices
        training_process = []
        for i in tqdm(range(self.iterations), total=self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i == 0) or ((i + 1) % (self.iterations / 10) == 0):
                print("Iteration: %d ; error = %.4f" % (i + 1, mse))

        return training_process

    def mse(self):
        """
        Mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error) / len(self.R)

    def sgd(self):
        """
        Stochastic gradient descent
        """
        for i, j, r in self.samples:
            # calculate the prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # update user and item hidden feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

    def get_rating(self, i, j):
        """
        Get preficted score (r_ij)，where i is the user id and j denotes item id
        """
        prediction = self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Obtain the fully prediction matrix
        """
        return self.P.dot(self.Q.T)

### 3. Training data

In [75]:
#Train the MF model
mf = MF(R, K=20, alpha=0.01, beta=0.3, iterations=5)
mf.train()

# ------ Recommendation example ------ #
# Recommend top-10 items to user_1
each_user = 1
user_ratings = mf.full_matrix()[each_user].tolist()
topN = [(i, user_ratings.funny(i)) for i in user_ratings]  # Correlate the item id with corresponding scores
topN = [i[1] for i in sorted(topN, key=lambda x: x[0], reverse=True)][:10]  # select top-10 items

print("------ user ------")
print(each_user)
print("------ temp_topN ------")
print(topN)


 20%|██        | 1/5 [00:01<00:04,  1.09s/it]

Iteration: 1 ; error = 0.9154


 40%|████      | 2/5 [00:02<00:03,  1.08s/it]

Iteration: 2 ; error = 0.4114


 60%|██████    | 3/5 [00:03<00:02,  1.07s/it]

Iteration: 3 ; error = 0.3499


 80%|████████  | 4/5 [00:04<00:01,  1.07s/it]

Iteration: 4 ; error = 0.3310


100%|██████████| 5/5 [00:05<00:00,  1.07s/it]

Iteration: 5 ; error = 0.3226
------ user ------
1
------ temp_topN ------
[408, 318, 483, 64, 603, 169, 302, 114, 285, 272]


### 4. evaluate with test data

In [76]:
# evaluate with test data
R = getUIMat(get_data(staticDir + '/ua.test'))
mf = MF(R, K=20, alpha=0.01, beta=0.3, iterations=5)
mf.train()
# ------ Recommendation example ------ #
# Recommend top-10 items to user_1
each_user = 1
user_ratings = mf.full_matrix()[each_user].tolist()
topN = [(i, user_ratings.funny(i)) for i in user_ratings]  # Correlate the item id with corresponding scores
topN = [i[1] for i in sorted(topN, key=lambda x: x[0], reverse=True)][:10]  # select top-10 items

print("------ user ------")
print(each_user)
print("------ temp_topN ------")
print(topN)

 40%|████      | 2/5 [00:00<00:00,  7.83it/s]

Iteration: 1 ; error = 0.3863
Iteration: 2 ; error = 0.3859


 80%|████████  | 4/5 [00:00<00:00,  6.84it/s]

Iteration: 3 ; error = 0.3854
Iteration: 4 ; error = 0.3846


100%|██████████| 5/5 [00:00<00:00,  7.00it/s]

Iteration: 5 ; error = 0.3830
------ user ------
1
------ temp_topN ------
[258, 50, 286, 181, 83, 172, 151, 173, 15, 554]


As we can see, the performance of the training and test sets is similar, indicating that there is no overfitting problem.

# Question 3 - Information Retrieval

In [10]:
import pandas
import numpy as np

1: construct a vocabulary table.

In [8]:
documents = [
    "big cats are nice and funny",
    "small dogs are better than big dogs",
    "small cats are afraid of small dogs",
    "big cats are not afraid of small dogs",
    "funny cats are not afraid of small dogs"
]
vocabs = [d.strip("-").strip(".").strip("?").strip(",") for doc in documents for d in doc.strip().split()]
vocabs = list(set(vocabs))
vocabs

['dogs',
 'of',
 'better',
 'big',
 'small',
 'are',
 'and',
 'afraid',
 'not',
 'cats',
 'funny',
 'nice',
 'than']

2:compute the one-hot vector

In [12]:
one_hot_vec = np.zeros((len(documents), len(vocabs)))
for i, doc in enumerate(documents):
    for d in doc.strip().split():
        one_hot_vec[i][vocabs.index(d)] = 1
one_hot_vec

array([[0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.],
       [1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.],
       [1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.],
       [1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.]])

3: retrieval each document which satifies Boolean Model.

In [19]:
rs_doc = np.array(["d1", "d2", "d3", "d4", "d5"])
# query_1 = funny AND dogs
funny_i = vocabs.index("funny")
dogs_i = vocabs.index("dogs")
q_1 = np.zeros((len(vocabs)))
q_1[funny_i], q_1[dogs_i] = 1, 1
np_sum_rs = np.sum(one_hot_vec * q_1, axis=1)
print("retrieval docuements for query_1: ", rs_doc[np.where(np_sum_rs >= 2)])

# query 2 = nice OR dog
nice_i = vocabs.index("nice")
q_2 = np.zeros((len(vocabs)))
q_2[nice_i], q_2[dogs_i] = 1, 1
np_sum_rs = np.sum(one_hot_vec * q_2, axis=1)
print("retrieval docuements for query_2: ", rs_doc[np.where(np_sum_rs >= 1)])

# query 3 = big AND dog AND NOT funny
big_i = vocabs.index("big")
n_funny_i = vocabs.index("funny")
q_3 = np.zeros((len(vocabs)))
q_3[n_funny_i], q_3[big_i], q_3[dogs_i] = -1, 1, 1
np_sum_rs = np.sum(one_hot_vec * q_3, axis=1)
print("retrieval docuements for query_3: ", rs_doc[np.where(np_sum_rs >= 2)])

retrieval docuements for query_1:  ['d5']
retrieval docuements for query_2:  ['d1' 'd2' 'd3' 'd4' 'd5']
retrieval docuements for query_3:  ['d2' 'd4']


### 3.2 Vector Space Model - TFIDF
Given a Wikipedia corpus and a set of queries, please find the top-5 most relevant documents for each query using the Tf-idf weighting scheme and Euclidean distance.

In [150]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import json

1: build up vocabulary each word.

In [131]:
wiki_corpus_lines = open('wiki_corpus.txt', encoding='UTF-8').readlines()
wiki_corpus = [lines.strip('\n') for lines in wiki_corpus_lines]
# print top 5 queries
wiki_corpus[:5]

['Both battalions attacked with over in waves of two companies each, at Without reconnaissance, the battalions ran into obstacles halfway to their objective.',
 'The bodies of the full-size dolls vary depending on the time of the release.',
 'The following player received entry using a protected ranking into the singles main draw:',
 "Its anchor tenants are a Safeway grocery store, a Longs Drugs store and the only McDonald's in the region.",
 'In 1993 Lotter toured with the Springboks to Australia, playing in two of the three tests.']

In [138]:
word_list = [words.strip(',').strip('.').split() for words in wiki_corpus]
word_list[:1]

[['Both',
  'battalions',
  'attacked',
  'with',
  'over',
  'in',
  'waves',
  'of',
  'two',
  'companies',
  'each,',
  'at',
  'Without',
  'reconnaissance,',
  'the',
  'battalions',
  'ran',
  'into',
  'obstacles',
  'halfway',
  'to',
  'their',
  'objective']]

2: build tf matrix.

In [140]:
class TF_IDF_Model(object):
    def __init__(self, documents_list):
        self.documents_list = documents_list
        self.documents_number = len(documents_list)
        self.tf = []
        self.idf = {}
        self.init()

    def init(self):
        df = {}
        for document in self.documents_list:
            # compute term frequency
            temp = {}
            for word in document:
                temp[word] = temp.get(word, 0) + 1/len(document)
            self.tf.append(temp)
            for key in temp.keys():
                # compute document frequency with term
                df[key] = df.get(key, 0) + 1
        # compute inverse document frequency with term
        for key, value in df.items():
            self.idf[key] = np.log(self.documents_number / (value + 1))

    def get_score(self, index, query):
        score = 0.0
        for q in query:
            if q not in self.tf[index]:
                continue
                # calculate score
            score += self.tf[index][q] * self.idf[q]
        return score

    def get_documents_score(self, query):
        score_list = []
        for i in range(self.documents_number):
            score_list.append(self.get_score(i, query))
        return score_list


In [147]:
tf_idf_model = TF_IDF_Model(word_list)
tf_idf_model.tf[:1]

[{'Both': 0.043478260869565216,
  'battalions': 0.08695652173913043,
  'attacked': 0.043478260869565216,
  'with': 0.043478260869565216,
  'over': 0.043478260869565216,
  'in': 0.043478260869565216,
  'waves': 0.043478260869565216,
  'of': 0.043478260869565216,
  'two': 0.043478260869565216,
  'companies': 0.043478260869565216,
  'each,': 0.043478260869565216,
  'at': 0.043478260869565216,
  'Without': 0.043478260869565216,
  'reconnaissance,': 0.043478260869565216,
  'the': 0.043478260869565216,
  'ran': 0.043478260869565216,
  'into': 0.043478260869565216,
  'obstacles': 0.043478260869565216,
  'halfway': 0.043478260869565216,
  'to': 0.043478260869565216,
  'their': 0.043478260869565216,
  'objective': 0.043478260869565216}]

In [158]:
def search_engine(query):
    score = tf_idf_model.get_documents_score(query)
    score = np.array(score)
    top_n = np.argsort(score)[::-1][:5]
    # format to json style
    ret = []
    for i in top_n:
        ret.append("d_" + str(i))
    jsonFormat = {}
    jsonFormat['query'] = query
    jsonFormat['results'] = ret
    with open('tfidf retrieval results.txt', 'a+') as write_f:
        write_f.write(json.dumps(jsonFormat, indent=4, ensure_ascii=True))

prepare for query

In [159]:
f = open('query.txt', 'r', encoding='UTF-8')
query = [line.strip('\n') for line in f.readlines()]
query[:5]

['Vowell is on the advisory board of 826NYC, a nonprofit tutoring and writing center for students aged 6–18 in Brooklyn.',
 'He authored a number of articles published in Russian and Georgian press, and discovered a hitherto unknown version of the medieval Georgian chronicle, "Moktsevay Kartlisay" (“The Conversion of Georgia”) (the so-called Chelishi codex).',
 'The Murphys turned over running of the rancho to their son Patrick Murphy, who was a General in the California National Guard.',
 'Several team names and spellings were altered during the 1990s when traditional Indian names were introduced to replace those that were associated with the British Raj.',
 'The S/L layouts had initially been based on a spacing of , but due to equipment shortages this had been extended to by September 1940.']

export output file

In [ ]:
for q in query:
    search_engine(q)

###  3.3
Given a Wikipedia corpus and a set of queries, please find the top-5 most relevant docu-ments for each query using the BM25.

#### import package

In [ ]:
! pip install rank-bm25

In [12]:
from rank_bm25 import BM25Okapi
import numpy as np

#### build up bm25 of corpus

In [13]:
wiki_corpus_lines = list(open('wiki_corpus.txt', 'r', encoding='UTF-8'))
corpus = [doc.split(" ") for doc in wiki_corpus_lines]
# build up a bm25 model
bm25 = BM25Okapi(corpus)

#### build up search engine using rank_bm25

In [19]:



def search_engin_bm25(query):
    tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    # get indexes
    top_n = np.argsort(scores)[::-1][:5]
    # format to json style
    ret = []
    for i in top_n:
        ret.append("d_" + str(i))
    jsonFormat = {}
    jsonFormat['query'] = query
    jsonFormat['results'] = ret
    with open('bm25 retrieval results.txt', 'a+') as write_f:
        write_f.write(json.dumps(jsonFormat, indent=4, ensure_ascii=True))

#### export output file

In [ ]:
for q in query:
    search_engin_bm25(q)

The Result has been on txt files.